# Standard library: smart pointers

## Pointers to nothing

#### Using `0` or `NULLPTR` does not differentiate a null integer from a null pointer

In [1]:
#include <iostream>

In [2]:
void process( char * a_value ) { std::cout<<"process(char *): "<<a_value<<std::endl ; }

In [3]:
void process( int a_value )    { std::cout<<"process(int): "<<a_value<<std::endl ; }

In [4]:
void process( long a_value ) { std::cout<<"process(long): "<<a_value<<std::endl ; }

In [5]:
const int NULLPTR = 0 ;

In [6]:
process(0) ;

process(int): 0


In [7]:
process(NULLPTR) ;

process(int): 0


In [8]:
process(NULL) ;    // implementation dependent

process(long): 0


This can lead to unwanted behaviors in case of overloading. 

#### C ++ 11 introduces `nullptr`, convertible to any type of pointer

In [9]:
process(nullptr) ;

process(char *): 

From now on, any pointer whose value is not yet known should be initiated with `nullptr`.

## `shared_ptr`

* The easiest way to handle objects made with `new`... and the slowest.
* A kind of **guard** which is **counting the references**.
* Provides dereferencing operators **\*** and **->**. 

#### A copyable pointer

In [1]:
#include <memory>
#include <vector>
#include <string>
#include <iostream>

In [2]:
void print( std::shared_ptr<std::string> a_text )
 { std::cout<<(*a_text)<<std::endl ; }  

In [4]:
std::shared_ptr<std::string> text {new std::string("hello")} ;

In [4]:
print(text) ;

hello


#### Shared pointers are practical, but expensive

* They are doubled in size, compared to an ordinary pointer, because they also point to a *control block* which notably contains the current number of references to the pointed object.
* The creation of the first pointer to a given object implies the dynamic creation of the *control block* associated with the pointed object.
* **Increasing or decreasing the number of references should be done in a thread-safe manner**, so it is a bit slowed down.

## `std::unique_ptr`

* The most efficient way to handle objects made with `new`.
* A kind of guard which is **moving the ownership**.
* Provides dereferencing operators **\*** and **->**. 

#### Difficulty: it is "move-only"

In [14]:
#include <memory>
#include <vector>
#include <string>
#include <iostream>

In [15]:
void print_val( std::unique_ptr<std::string> a_text )
 { std::cout<<(*a_text)<<std::endl ; }  

In [5]:
std::unique_ptr<std::string> text {new std::string("hello")} ;

In [17]:
print_val(text) ;

input_line_23:2:12: error: call to deleted constructor of 'std::unique_ptr<std::string>' (aka 'unique_ptr<basic_string<char> >')
 print_val(text) ;
           ^~~~
/opt/conda/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/unique_ptr.h:414:7: note: 'unique_ptr' has been explicitly marked deleted here
      unique_ptr(const unique_ptr&) = delete;
      ^
input_line_21:1:46: note: passing argument to parameter 'text' here
void print_val( std::unique_ptr<std::string> text )
                                             ^


Interpreter Error: 

In [18]:
void print_ref( std::unique_ptr<std::string> const & a_text )
 { std::cout<<(*a_text)<<std::endl ; }  

In [19]:
print_ref(text) ;

hello


#### Yet, usable in a collection

In [20]:
#include <memory>
#include <vector>
#include <string>
#include <iostream>

In [21]:
std::vector<std::unique_ptr<std::string>> words ;

words.push_back(std::unique_ptr<std::string>(new std::string("hello"))) ;
words.push_back(std::unique_ptr<std::string>(new std::string("world"))) ;
words.push_back(std::unique_ptr<std::string>(new std::string("!"))) ;

for ( auto const & word : words )
 { std::cout<<(*word)<<" " ; }

hello world ! 

The uniques pointers above are made on the fly, i.e. temporary, i.e. right values. Therefore, they can be **moved** into the vector.

In the range-based loop, do not forget the **&**, or the compiler will try to copy the unique pointers when reading them, and fail.

## `make_unique` and `make_shared`

#### Usual trap: giving a raw pointer to several smart pointers

In [ ]:
#include <iostream>
#include <string>
#include <memory>

In [ ]:
{
//...
int * ip = new int {1} ;
//...
std::shared_ptr<int> sp1 {ip} ;
//...
std::shared_ptr<int> sp2 {ip} ;
//...
}

#### Instead, give the result of `new` directly to the smart pointer

In [ ]:
#include <iostream>
#include <string>
#include <memory>

In [1]:
{
  //...
  std::shared_ptr<int> sp1 {new int {1}} ;
  //...
  std::shared_ptr<int> sp2 {sp1} ;
  //...
}

#### Even better: use `make_shared` and `make_unique`

In [ ]:
#include <iostream>
#include <string>
#include <memory>

In [3]:
{
  //...
  auto up1 = std::make_unique<int>(1) ;
  //...
  auto & up2 = up1 ;
  //...
  //...
  auto sp1 = std::make_shared<int>(1) ;
  //...
  auto sp2 = sp1 ;
  //...
}

## Questions ?

# Exercise

Eliminate the raw pointers from the example, and use smart pointers instead, so that the explicit call to `delete` in `main()` can be removed.

In [1]:
%%file tmp.pointers.cpp

#include <iostream>

class MyData
 {
  public :
    MyData( int a_data ) : m_data(a_data)
     { std::cout<<"MyData::MyData("<<m_data<<")"<<std::endl ; }
    int data() const { return m_data ; }
    ~MyData() { std::cout<<"MyData::~MyData("<<m_data<<")"<<std::endl ; }
  private :
    int m_data ;
 } ;

void print( MyData const * a_data_ptr )
 { std::cout<<a_data_ptr->data()<<std::endl ; }  

int main()
 {
  MyData * data_ptr = new MyData(42) ;
  print(data_ptr) ;
  delete data_ptr ;
  return 0 ;
 }

Writing tmp.pointers.cpp


In [ ]:
!rm -f tmp.pointers.exe && g++ -std=c++17 tmp.pointers.cpp -o tmp.pointers.exe

In [ ]:
!./tmp.pointers.exe

© *CNRS 2021*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*